### Books Recommendation System using Clustering | Collaborative Filtering

In [7]:
import pandas as pd
import numpy as np
import csv

In [8]:
books = pd.read_csv('data//BX-Books.csv', sep=';', encoding='latin-1', quoting=1, dtype=str, error_bad_lines=False)

books.head(2)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [9]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [10]:
books_final = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [11]:
books_final.head(2)	

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [12]:
books_final.rename(columns={'Book-Title': 'title', 'Book-Author': 'author', 'Year-Of-Publication': 'year', 'Publisher': 'publisher', 'Image-URL-L': 'image_url'}, inplace=True)

f:\MLOps\MLOps\Book-Recommendation-System-Azure\venv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
books_final.head()

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [14]:
users = pd.read_csv('data//BX-Users.csv', sep=';', encoding='latin-1', quoting=1, dtype=str, error_bad_lines=False)
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18


In [15]:
users.shape

(278858, 3)

In [16]:
ratings = pd.read_csv('data//BX-Book-Ratings.csv', sep=';', encoding='latin-1', quoting=1, dtype=str, error_bad_lines=False)
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [17]:
ratings.shape

(1149780, 3)

In [18]:
print(books_final.shape)
print(users.shape)
print(ratings.shape)

(271360, 6)
(278858, 3)
(1149780, 3)


In [19]:
ratings.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating'}, inplace=True)

ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [20]:
users.rename(columns={'User-ID': 'user_id', 'Location': 'location', 'Age': 'age'}, inplace=True)

users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18


In [21]:
ratings['user_id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
62286         1
197194        1
33365         1
43540         1
153897        1
Name: user_id, Length: 105283, dtype: int64

In [22]:
x = ratings['user_id'].value_counts() > 200

In [23]:
y = x[x].index

In [24]:
ratings = ratings[ratings['user_id'].isin(y)]

In [25]:
ratings.head(2)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0


In [26]:
ratings.shape

(526356, 3)

In [27]:
rating_with_books = ratings.merge(books_final, on='ISBN')
rating_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [28]:
rating_with_books.shape

(487671, 8)

In [29]:
num_rating = rating_with_books.groupby('title')['rating'].count().reset_index()

In [30]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [31]:
num_rating.rename(columns={'rating': 'num_ratings'}, inplace=True)

In [32]:
num_rating.head(2)

,title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1


In [33]:
final_ratings = rating_with_books.merge(num_rating, on='title')

In [34]:
final_ratings.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [35]:
final_rating = final_ratings[final_ratings['num_ratings'] >= 50]

In [36]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_ratings
105863,275970,0860074382,10,84 Charing Cross Road,Helene Hanff,0,Warner Books> C/o Little Br,http://images.amazon.com/images/P/0860074382.0...,54
75507,271284,0345386108,0,Winter Moon,Dean R. Koontz,1995,Ballantine Books,http://images.amazon.com/images/P/0345386108.0...,61
23233,204864,0141301066,10,Matilda,Roald Dahl,1998,Puffin Books,http://images.amazon.com/images/P/0141301066.0...,64
53926,173684,0375823468,10,"The Subtle Knife (His Dark Materials, Book 2)",PHILIP PULLMAN,2002,Knopf Books for Young Readers,http://images.amazon.com/images/P/0375823468.0...,83
93564,254465,0671894455,0,FORREST GUMP (Movie Tie in),Winston Groom,1994,Pocket,http://images.amazon.com/images/P/0671894455.0...,70
110011,189334,0452269571,0,Bastard Out of Carolina,Dorothy Allison,1993,Plume Books,http://images.amazon.com/images/P/0452269571.0...,62
115519,149934,0671014196,0,The Summerhouse,Jude Deveraux,2002,Pocket,http://images.amazon.com/images/P/0671014196.0...,84
53699,238557,0375727345,0,House of Sand and Fog,Andre Dubus III,2000,Vintage Books,http://images.amazon.com/images/P/0375727345.0...,174
160241,194669,0373483503,0,Macgregor Brides (Macgregors),Nora Roberts,1997,Silhouette,http://images.amazon.com/images/P/0373483503.0...,52
55553,59172,0451190750,9,Thinner,Stephen King,1996,Signet Book,http://images.amazon.com/images/P/0451190750.0...,81


In [37]:
final_rating.shape

(61853, 9)

In [38]:
final_rating.drop_duplicates(['title', 'user_id'], inplace=True)

f:\MLOps\MLOps\Book-Recommendation-System-Azure\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
final_rating.shape

(59850, 9)

In [40]:
final_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59850 entries, 0 to 236705
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59850 non-null  object
 1   ISBN         59850 non-null  object
 2   rating       59850 non-null  object
 3   title        59850 non-null  object
 4   author       59850 non-null  object
 5   year         59850 non-null  object
 6   publisher    59850 non-null  object
 7   image_url    59850 non-null  object
 8   num_ratings  59850 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 4.6+ MB


In [41]:
final_rating['rating'] = pd.to_numeric(final_rating['rating'], errors='coerce')


f:\MLOps\MLOps\Book-Recommendation-System-Azure\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
book_pivot = final_rating.pivot_table(index='title', columns='user_id', values='rating')

In [43]:
book_pivot.head(2)

user_id,100459,100644,100846,100906,101209,101305,101851,101876,102275,102359,...,95932,95991,96054,96448,97874,98297,98391,9856,98741,98758
title,,,,,,,,,,,,,,,,,,,,,
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
book_pivot.fillna(0, inplace = True)

In [45]:
book_pivot.head(1)

user_id,100459,100644,100846,100906,101209,101305,101851,101876,102275,102359,...,95932,95991,96054,96448,97874,98297,98391,9856,98741,98758
title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
from scipy.sparse import csr_matrix

In [47]:
book_sparse = csr_matrix(book_pivot)

In [48]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [49]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [50]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [51]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors = 6)

In [52]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [53]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [54]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [55]:
books_name = book_pivot.index

In [56]:
import pickle
pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(books_name, open('artifacts/books_name.pkl', 'wb'))
pickle.dump(final_rating, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [57]:
def recomment_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors = 6)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        
        for j in books:
            print(j)

In [58]:
recomment_book('Beach House')

Beach House
No Safe Place
Jacob Have I Loved
Last Man Standing
Long After Midnight
Exclusive


In [59]:
!pip show scipy

Name: scipy
Version: 1.7.3
Summary: SciPy: Scientific Library for Python
Home-page: https://www.scipy.org
Author: 
Author-email: 
License: BSD
Location: f:\mlops\mlops\book-recommendation-system-azure\venv\lib\site-packages
Requires: numpy
Required-by: scikit-learn
